In [122]:
from torch_geometric.data import InMemoryDataset, Data, Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import Sequential, GINConv, GATConv, global_mean_pool, global_max_pool, global_add_pool
from torch_geometric.utils import to_networkx, from_networkx, add_self_loops

from torch.nn import Linear, BatchNorm1d, ReLU, Dropout, LeakyReLU, SiLU, PReLU
import torch.nn.functional as F
import torch

import numpy as np
import networkx as nx
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix


from nba_api.stats.static import teams,players

import traceback
import difflib
import os

In [123]:
try:
    df_nba_teams = pd.read_csv('nba_teams_id.csv')
except:
    # Get all the NBA teams description
    nba_teams = teams.get_teams()
    df_nba_teams = pd.DataFrame(nba_teams)
    df_nba_teams.to_csv('nba_teams_id.csv',index=False)

df_nba_teams.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [124]:
team_id = 1610612741
df_nba_teams.query(f"id == {team_id}")['full_name'].values[0]

'Chicago Bulls'

In [125]:
# Get all the NBA players description
try:
    df_nba_players = pd.read_csv('nba_players.csv')
except:
    df_nba_players = pd.DataFrame(players.get_players())
    df_nba_players.to_csv('nba_players.csv',index=False)

df_nba_players.head()

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False


In [126]:
df_ratings_NBA2k_2020_2023 = pd.read_csv('ratings_NBA2k_2020_2023.csv')
df_ratings_NBA2k_2020_2023.head()

,Unnamed: 0,Season_year,Team,Player,Shot Close,Shot Mid,Shot 3pt,Shot IQ,Free Throw,Offensive Consistency,...,Help Defense IQ,Lateral Quickness,Pass Perception,Steal,Block,Defensive Consistency,Offensive Rebound,Defensive Rebound,Intangibles,Potential
0,0,2020,Atlanta Hawks,Trae Young,83,77,80,90,81,96,...,56,68,68,39,35,28,36,48,98,89
1,1,2020,Atlanta Hawks,John Collins,69,67,72,90,70,89,...,68,51,32,26,51,54,80,75,95,86
2,2,2020,Atlanta Hawks,Jabari Parker,81,70,69,90,66,75,...,64,62,63,36,50,50,44,75,95,78
3,3,2020,Atlanta Hawks,Kevin Huerter,72,70,80,90,71,63,...,61,70,62,41,49,33,37,47,95,82
4,4,2020,Atlanta Hawks,De'Andre Hunter,62,71,74,90,70,42,...,72,80,52,32,46,72,47,51,95,88


In [127]:
df_players_formation = pd.read_csv('players_formation.csv')
df_players_formation.head()

,Unnamed: 0,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,START_POSITION
0,0,22001066,1610612745,1630256,Jae'Sean Tate,F
1,1,22001066,1610612745,1630231,Kenyon Martin Jr.,F
2,2,22001066,1610612745,203482,Kelly Olynyk,C
3,3,22001066,1610612745,1630237,Anthony Lamb,G
4,4,22001066,1610612745,201571,D.J. Augustin,G


In [128]:
with open('season_campaign_all_teams_dict.pickle','rb') as results:
    season_campaign_all_teams_dict = pickle.load(results)

season_campaign_all_teams_dict['2020-21'][1610612766][1610612766]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,TEAM_ADVERSARY_ABBREVIATION,WINNER
0,22020,1610612766,CHA,Charlotte Hornets,0022001080,2021-05-16,CHA @ WAS,L,240,110,...,32,42,25,8,5,11,15,-5.0,WAS,False
1,22020,1610612766,CHA,Charlotte Hornets,0022001064,2021-05-15,CHA @ NYK,L,265,109,...,36,51,28,5,2,8,18,-9.0,NYK,False
2,22020,1610612766,CHA,Charlotte Hornets,0022001047,2021-05-13,CHA vs. LAC,L,239,90,...,27,35,24,9,1,10,15,-23.0,LAC,False
3,22020,1610612766,CHA,Charlotte Hornets,0022000448,2021-05-11,CHA vs. DEN,L,240,112,...,34,49,30,9,5,17,17,-5.0,DEN,False
4,22020,1610612766,CHA,Charlotte Hornets,0022001020,2021-05-09,CHA vs. NOP,L,240,110,...,35,46,22,6,8,17,19,-2.0,NOP,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,22020,1610612766,CHA,Charlotte Hornets,0022000069,2021-01-01,CHA vs. MEM,L,241,93,...,32,42,23,9,4,18,15,-15.0,MEM,False
68,22020,1610612766,CHA,Charlotte Hornets,0022000059,2020-12-30,CHA @ DAL,W,240,118,...,38,50,30,6,6,23,28,19.0,DAL,True
69,22020,1610612766,CHA,Charlotte Hornets,0022000032,2020-12-27,CHA vs. BKN,W,241,106,...,39,52,35,8,3,12,25,2.0,BKN,True
70,22020,1610612766,CHA,Charlotte Hornets,0022000022,2020-12-26,CHA vs. OKC,L,239,107,...,34,47,27,11,8,13,22,-2.0,OKC,False


In [129]:
count=0
for y,d1 in season_campaign_all_teams_dict.items():
    for k1,d2 in d1.items():
        for k2,d3 in d2.items():
            count+=1
            # print(y,k2)
count


90

In [130]:
df_ratings_NBA2k_2020_2023.head()

,Unnamed: 0,Season_year,Team,Player,Shot Close,Shot Mid,Shot 3pt,Shot IQ,Free Throw,Offensive Consistency,...,Help Defense IQ,Lateral Quickness,Pass Perception,Steal,Block,Defensive Consistency,Offensive Rebound,Defensive Rebound,Intangibles,Potential
0,0,2020,Atlanta Hawks,Trae Young,83,77,80,90,81,96,...,56,68,68,39,35,28,36,48,98,89
1,1,2020,Atlanta Hawks,John Collins,69,67,72,90,70,89,...,68,51,32,26,51,54,80,75,95,86
2,2,2020,Atlanta Hawks,Jabari Parker,81,70,69,90,66,75,...,64,62,63,36,50,50,44,75,95,78
3,3,2020,Atlanta Hawks,Kevin Huerter,72,70,80,90,71,63,...,61,70,62,41,49,33,37,47,95,82
4,4,2020,Atlanta Hawks,De'Andre Hunter,62,71,74,90,70,42,...,72,80,52,32,46,72,47,51,95,88


In [131]:
def get_team_name(team_id):
    return df_nba_teams.query(f"id == {team_id}")['full_name'].values[0]

def create_fully_connected_graph_new(G_completed_team, game_id, team_id, show_graph, season_year, team_name, df_ratings_NBA2k_2020_2023_PLAYER_ID):

    df_rost = df_players_formation.query(f"GAME_ID == {int(game_id)} & TEAM_ID == {team_id}")[['PLAYER_NAME','START_POSITION']]

    reserve_players = df_rost['PLAYER_NAME'][df_rost['START_POSITION'].isna().to_list()].values
    starting_lineup_players = df_rost['PLAYER_NAME'][np.invert(df_rost['START_POSITION'].isna().values)].values

    rating_embeddings_new = []
    for player_name in reserve_players:
        embedding = rating_embeddings(df_ratings_NBA2k_2020_2023_PLAYER_ID,season_year,team_name,player_name)

        rating_embeddings_new.append((player_name,{'x':embedding,'starting_lineup':False}))

    G_completed_team.add_nodes_from(rating_embeddings_new)


    rating_embeddings_new = []
    for player_name in starting_lineup_players:
        embedding = rating_embeddings(df_ratings_NBA2k_2020_2023_PLAYER_ID,season_year,team_name,player_name)
        rating_embeddings_new.append((player_name,{'x':embedding,'starting_lineup':True}))

    G_completed_team.add_nodes_from(rating_embeddings_new)


    # assert sum(nx.get_node_attributes(G_completed_team,'starting_lineup').values()) == 5, "should be 5 players is in the stating lineup"

    G_completed_team_full_connected = nx.complete_graph(G_completed_team.nodes)
    G_completed_team = nx.compose(G_completed_team, G_completed_team_full_connected)

    # nx.draw(G_completed_team, with_labels=True, font_weight='bold',node_color="red")

    G_completed_team = nx.compose(G_completed_team, G_completed_team_full_connected)

    if (show_graph):
        starting_lineup_colors = list(nx.get_node_attributes(G_completed_team,'starting_lineup').values())
        nx.draw_circular(G_completed_team, with_labels=True, font_weight='bold',node_color=['red' if x else 'blue' for x in starting_lineup_colors])
        plt.show()

    return G_completed_team, starting_lineup_players, df_rost

def create_fully_connected_graph(G_completed_team, game_id, team_id, show_graph, season_year, team_name):

    player_name=player_name
    
    
    df_rost = df_players_formation.query(f"GAME_ID == {int(game_id)} & TEAM_ID == {team_id}")[['PLAYER_NAME','START_POSITION']]

    reserve_players = df_rost['PLAYER_NAME'][df_rost['START_POSITION'].isna().to_list()].values
    starting_lineup_players = df_rost['PLAYER_NAME'][np.invert(df_rost['START_POSITION'].isna().values)].values
    G_completed_team.add_nodes_from(reserve_players,starting_lineup=False)
    G_completed_team.add_nodes_from(starting_lineup_players,starting_lineup=True)


    # assert sum(nx.get_node_attributes(G_completed_team,'starting_lineup').values()) == 5, "should be 5 players is in the stating lineup"

    G_completed_team_full_connected = nx.complete_graph(G_completed_team.nodes)
    G_completed_team = nx.compose(G_completed_team, G_completed_team_full_connected)

    # nx.draw(G_completed_team, with_labels=True, font_weight='bold',node_color="red")

    G_completed_team = nx.compose(G_completed_team, G_completed_team_full_connected)

    if (show_graph):
        starting_lineup_colors = list(nx.get_node_attributes(G_completed_team,'starting_lineup').values())
        nx.draw_circular(G_completed_team, with_labels=True, font_weight='bold',node_color=['red' if x else 'blue' for x in starting_lineup_colors])
        plt.show()

    return G_completed_team, starting_lineup_players



# check what payers in ratings_NBA2k_2020_2023.csv doesn't have the respective PLAYER_ID of the players_formation.csv
# df_ratings_NBA2k_2020_2023_PLAYER_ID[df_ratings_NBA2k_2020_2023_PLAYER_ID['PLAYER_ID'].map(lambda x:np.isnan(x))]

df_ratings_NBA2k_2020_2023_PLAYER_ID = pd.merge(df_ratings_NBA2k_2020_2023,df_nba_players[['full_name','id']], how='left', copy=False, left_on='Player',right_on='full_name')
def rating_embeddings(df_ratings_NBA2k_2020_2023_PLAYER_ID, season_year,team_name,player_name):
    rating_columns = ['Shot Close', 'Shot Mid','Shot 3pt', 'Shot IQ', 'Free Throw', 'Offensive Consistency','Driving Layup', 'Standing Dunk', 'Driving Dunk', 'Draw Foul','Post Moves', 'Post Hook', 'Post Fade', 'Hands', 'Speed With Ball','Ball Handle', 'Passing Accuracy', 'Passing Vision', 'Passing IQ','Speed', 'Acceleration', 'Vertical', 'Strength', 'Stamina', 'Hustle','Interior Defense', 'Perimeter Defense', 'Help Defense IQ','Lateral Quickness', 'Pass Perception', 'Steal', 'Block','Defensive Consistency', 'Offensive Rebound', 'Defensive Rebound','Intangibles', 'Potential']
    
    try:
        # return torch.tensor(df_ratings_NBA2k_2020_2023_PLAYER_ID.query(f"Season_year == {season_year} and Team == '{team_name}' and Player == '{player_name}'")[rating_columns].values[0])
        return df_ratings_NBA2k_2020_2023_PLAYER_ID.query(f"Season_year == {season_year} and Team == '{team_name}' and Player == '{player_name}'")[rating_columns].values[0]
    except:
        pass
    try:
        # return torch.tensor(df_ratings_NBA2k_2020_2023_PLAYER_ID.query(f"Team == '{team_name}' and Player == '{player_name}'")[rating_columns].values[0])
        return df_ratings_NBA2k_2020_2023_PLAYER_ID.query(f"Team == '{team_name}' and Player == '{player_name}'")[rating_columns].values[0]
    except:
        pass

    # try the closest name
    try:
        player_name_closed = difflib.get_close_matches(player_name,df_ratings_NBA2k_2020_2023_PLAYER_ID['Player'],1)[0]
        # return torch.tensor(df_ratings_NBA2k_2020_2023_PLAYER_ID.query(f"Player == '{player_name_closed}'")[rating_columns].values[0])
        return df_ratings_NBA2k_2020_2023_PLAYER_ID.query(f"Player == '{player_name_closed}'")[rating_columns].values[0]
    except:
        pass

    # return zeros if the player cannot be found
    return np.zeros([len(rating_columns)], dtype=np.int64)

def train(model, data_train_loader,criterion, optimizer, device='cpu'):
    try:
        for data_train in data_train_loader:
            # data_train = data_train.to(device)
            optimizer.zero_grad()  # Clear gradients.
            out = model(
                x=data_train.x,
                edge_index=data_train.edge_index,
                batch=data_train.batch,
                device=device
            )
        
            loss = criterion(out, data_train.y)
            # print(loss)
            loss.backward()  # Derive gradients.
            optimizer.step()  # Update parameters based on gradients.
    except:
        traceback.print_exc()
        
def test(model, data_train_loader,criterion, device='cpu'):
    pred_y_total = torch.tensor([], device=device)
    actual_y_total = torch.tensor([], device=device)
    valid_losses = []
    for data_train in data_train_loader:
        # data_train = data_train.to(device)
        out = model(
            x=data_train.x,
            edge_index=data_train.edge_index,
            batch=data_train.batch,
            device=device
        )


        pred_y = out.argmax(dim=1)  # Use the class with highest probability.
        pred_y_total = torch.cat([pred_y_total,pred_y])        
        actual_y = data_train.y.squeeze()       
        actual_y_total = torch.cat([actual_y_total,actual_y])


        loss = criterion(out, data_train.y)
        valid_losses.append(loss.item()) # record validation loss

    valid_losses = np.average(valid_losses)
    
    actual_y_total_cpu = actual_y_total.cpu()
    pred_y_total_cpu = pred_y_total.cpu()
    acc = accuracy_score(y_true=actual_y_total_cpu, y_pred=pred_y_total_cpu)
    precision = precision_score(y_true=actual_y_total_cpu, y_pred=pred_y_total_cpu, average=None, zero_division=0)
    precision_avg = precision_score(y_true=actual_y_total_cpu, y_pred=pred_y_total_cpu, average='weighted', zero_division=0)
    recall = recall_score(y_true=actual_y_total_cpu, y_pred=pred_y_total_cpu, average=None,zero_division=0)        
    recall_avg = recall_score(y_true=actual_y_total_cpu, y_pred=pred_y_total_cpu, average='weighted',zero_division=0)
    f1_score_result = f1_score(y_true=actual_y_total_cpu, y_pred=pred_y_total_cpu, average=None, zero_division=0)
    f1_score_result_avg = f1_score(y_true=actual_y_total_cpu, y_pred=pred_y_total_cpu, average='weighted', zero_division=0)
    c_matrix = confusion_matrix(y_true=actual_y_total_cpu, y_pred=pred_y_total_cpu)

    return acc, valid_losses, precision, precision_avg, recall, recall_avg, f1_score_result, f1_score_result_avg, c_matrix, actual_y_total_cpu, pred_y_total_cpu
    


In [132]:
class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, dataset_name, season_list, df_nba_teams, season_campaign_all_teams_dict, transform=None, pre_transform=None, pre_filter=None):
        # Initialize the dataset with some arguments
        self.dataset_name = dataset_name
        self.season_list = season_list
        self.df_nba_teams = df_nba_teams
        self.season_campaign_all_teams_dict = season_campaign_all_teams_dict
        super(MyOwnDataset, self).__init__(root, transform, pre_transform,pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
    
    @property
    def raw_file_names(self):
        # Return a list of the names of the raw data files
        # return os.listdir(self.raw_dir)     
        return ['file_gnn']  

    @property
    def processed_file_names(self):
        # Return a list of the names of the processed data files
        return [f'{self.dataset_name}_{i}.pt' for i in range(len(self.raw_file_names))]

    def process(self):
        # Process your raw data files and convert them to Data objects
        # Save the processed data to disk

        verbose = False
        G_completed_team_1_2_list = []
        for season in self.season_list:
            print(season)
            for team_1_id, team_1_name in zip(self.df_nba_teams['id'],self.df_nba_teams['full_name']):        
                
                df_season_campaign_all_teams_dict = self.season_campaign_all_teams_dict[season][team_1_id][team_1_id]

                for team_2,game_id,winner in zip(df_season_campaign_all_teams_dict['TEAM_ADVERSARY_ABBREVIATION'],df_season_campaign_all_teams_dict['GAME_ID'],df_season_campaign_all_teams_dict['WINNER']):
                    team_2_id = self.df_nba_teams.query(f"abbreviation == '{team_2}'")['id'].values[0]

                    season_year=int(season[:4])

                    G_completed_team_1 = nx.Graph()
                    G_completed_team_1, starting_lineup_players_team_1, df_rost_new = create_fully_connected_graph_new(G_completed_team=G_completed_team_1, game_id=game_id, team_id=team_1_id, show_graph=verbose, season_year=season_year,team_name=team_1_name,df_ratings_NBA2k_2020_2023_PLAYER_ID=df_ratings_NBA2k_2020_2023_PLAYER_ID)

                    G_completed_team_2 = nx.Graph()
                    team_2_name = get_team_name(team_2_id)
                    G_completed_team_2, starting_lineup_players_team_2, df_rost_new = create_fully_connected_graph_new(G_completed_team=G_completed_team_2, game_id=game_id, team_id=team_2_id, show_graph=verbose, season_year=season_year,team_name=team_2_name,df_ratings_NBA2k_2020_2023_PLAYER_ID=df_ratings_NBA2k_2020_2023_PLAYER_ID)
                    
                    G_completed_team_1_2 = nx.compose(G_completed_team_1,G_completed_team_2)

                    if (verbose):
                        starting_lineup_colors = list(nx.get_node_attributes(G_completed_team_1_2,'starting_lineup').values())                    
                        nx.draw_circular(G_completed_team_1_2, with_labels=True, font_weight='bold',node_color=['red' if x else 'blue' for x in starting_lineup_colors])
                        plt.show()

                    starting_lineup_players_fully_connected = G_completed_team_1_2.subgraph(np.concatenate([starting_lineup_players_team_1,starting_lineup_players_team_2]))
                    starting_lineup_players_fully_connected = nx.complete_graph(starting_lineup_players_fully_connected)

                    G_completed_team_1_2 = nx.compose(G_completed_team_1_2, starting_lineup_players_fully_connected)
                    if (verbose):
                        starting_lineup_colors = list(nx.get_node_attributes(G_completed_team_1_2,'starting_lineup').values())
                        nx.draw_circular(G_completed_team_1_2, with_labels=True, font_weight='bold',node_color=['red' if x else 'blue' for x in starting_lineup_colors])
                        plt.show()

                    graph_instance = from_networkx(G_completed_team_1_2)                    
                    graph_instance.y = int(winner)
                    # update the list of instances
                    G_completed_team_1_2_list.append(graph_instance)

                    # print(f"Season: {season_year} | {team_1_id} : {team_1_name} | {team_2_id} : {team_2_name}")        

        # if self.pre_filter is not None:
        #     self.data = [data for data in self.data if self.pre_filter(data)]

        # if self.pre_transform is not None:
        #     self.data = [self.pre_transform(data) for data in self.data]

        data, slices = self.collate(G_completed_team_1_2_list)

        torch.save((data, slices), self.processed_paths[0])

    # def len(self):
    #     # Return the number of graphs in the dataset
    #     return len(self.processed_file_names)

    # def get(self, idx):
    #     # Return the Data object at index idx
    #     data = torch.load(os.path.join(self.processed_dir, f'{self.dataset_name}_{idx}.pt'))
    #     return data



'''
verbose = False
G_completed_team_1_2_list = []
for season in season_list:
    print(season)
    for team_1_id, team_1_name in zip(df_nba_teams['id'],df_nba_teams['full_name']):        
        
        df_season_campaign_all_teams_dict = season_campaign_all_teams_dict[season][team_1_id][team_1_id]

        for team_2,game_id,winner in zip(df_season_campaign_all_teams_dict['TEAM_ADVERSARY_ABBREVIATION'],df_season_campaign_all_teams_dict['GAME_ID'],df_season_campaign_all_teams_dict['WINNER']):
            team_2_id = df_nba_teams.query(f"abbreviation == '{team_2}'")['id'].values[0]

            season_year=int(season[:4])

            G_completed_team_1 = nx.Graph()
            G_completed_team_1, starting_lineup_players_team_1, df_rost_new = create_fully_connected_graph_new(G_completed_team=G_completed_team_1, game_id=game_id, team_id=team_1_id, show_graph=verbose, season_year=season_year,team_name=team_1_name,df_ratings_NBA2k_2020_2023_PLAYER_ID=df_ratings_NBA2k_2020_2023_PLAYER_ID)

            G_completed_team_2 = nx.Graph()
            team_2_name = get_team_name(team_2_id)
            G_completed_team_2, starting_lineup_players_team_2, df_rost_new = create_fully_connected_graph_new(G_completed_team=G_completed_team_2, game_id=game_id, team_id=team_2_id, show_graph=verbose, season_year=season_year,team_name=team_2_name,df_ratings_NBA2k_2020_2023_PLAYER_ID=df_ratings_NBA2k_2020_2023_PLAYER_ID)
            
            G_completed_team_1_2 = nx.compose(G_completed_team_1,G_completed_team_2)

            if (verbose):
                starting_lineup_colors = list(nx.get_node_attributes(G_completed_team_1_2,'starting_lineup').values())                    
                nx.draw_circular(G_completed_team_1_2, with_labels=True, font_weight='bold',node_color=['red' if x else 'blue' for x in starting_lineup_colors])
                plt.show()

            starting_lineup_players_fully_connected = G_completed_team_1_2.subgraph(np.concatenate([starting_lineup_players_team_1,starting_lineup_players_team_2]))
            starting_lineup_players_fully_connected = nx.complete_graph(starting_lineup_players_fully_connected)

            G_completed_team_1_2 = nx.compose(G_completed_team_1_2, starting_lineup_players_fully_connected)
            if (verbose):
                starting_lineup_colors = list(nx.get_node_attributes(G_completed_team_1_2,'starting_lineup').values())
                nx.draw_circular(G_completed_team_1_2, with_labels=True, font_weight='bold',node_color=['red' if x else 'blue' for x in starting_lineup_colors])
                plt.show()

            graph_instance = from_networkx(G_completed_team_1_2)
            graph_instance.y = winner
            # update the list of instances
            G_completed_team_1_2_list.append(graph_instance)

            print(f"Season: {season_year} | {team_1_id} : {team_1_name} | {team_2_id} : {team_2_name}")        
            
        try:            
            pass    

        except:
            # print(f'The {team_1_id} does not have info')
            traceback.print_exc()
            # pass
'''


'\nverbose = False\nG_completed_team_1_2_list = []\nfor season in season_list:\n    print(season)\n    for team_1_id, team_1_name in zip(df_nba_teams[\'id\'],df_nba_teams[\'full_name\']):        \n        \n        df_season_campaign_all_teams_dict = season_campaign_all_teams_dict[season][team_1_id][team_1_id]\n\n        for team_2,game_id,winner in zip(df_season_campaign_all_teams_dict[\'TEAM_ADVERSARY_ABBREVIATION\'],df_season_campaign_all_teams_dict[\'GAME_ID\'],df_season_campaign_all_teams_dict[\'WINNER\']):\n            team_2_id = df_nba_teams.query(f"abbreviation == \'{team_2}\'")[\'id\'].values[0]\n\n            season_year=int(season[:4])\n\n            G_completed_team_1 = nx.Graph()\n            G_completed_team_1, starting_lineup_players_team_1, df_rost_new = create_fully_connected_graph_new(G_completed_team=G_completed_team_1, game_id=game_id, team_id=team_1_id, show_graph=verbose, season_year=season_year,team_name=team_1_name,df_ratings_NBA2k_2020_2023_PLAYER_ID=df_rating

In [133]:
# G_completed_team_1 = nx.Graph()
# game_id = '0022001080'
# team_id = 1610612766
# show_graph = verbose
# season_year = 2020
# team_name = 'Charlotte Hornets'
# G_completed_team_1, starting_lineup_players_team_1,df_rost_new = create_fully_connected_graph_new(G_completed_team=G_completed_team_1, game_id=game_id, team_id=team_1_id, show_graph=verbose, season_year=season_year,team_name=team_name,df_ratings_NBA2k_2020_2023_PLAYER_ID=df_ratings_NBA2k_2020_2023_PLAYER_ID)

# # for player_name in df_rost_new['PLAYER_NAME']:
# # player_name = 'Miles Bridges'
# # for player_name in df_rost_new['PLAYER_NAME']:
# #     try:
# #         print(rating_embeddings(df_ratings_NBA2k_2020_2023_PLAYER_ID, season_year,team_name,player_name))
# #         print(player_name, 'OK')
# #     except:
# #         print(player_name, 'Fail')
#         # traceback.print_exc()

# reserve_players = df_rost_new['PLAYER_NAME'][df_rost_new['START_POSITION'].isna().to_list()].values
# starting_lineup_players = df_rost_new['PLAYER_NAME'][np.invert(df_rost_new['START_POSITION'].isna().values)].values

In [134]:
# season_year = 2020
# team_name = "Atlanta Hawks"
# player_name = "Jabari Parker"


# # rating_embeddings(df_ratings_NBA2k_2020_2023_PLAYER_ID,season_year,team_name,player_name)
# rating_columns = ['Shot Close', 'Shot Mid','Shot 3pt', 'Shot IQ', 'Free Throw', 'Offensive Consistency','Driving Layup', 'Standing Dunk', 'Driving Dunk', 'Draw Foul','Post Moves', 'Post Hook', 'Post Fade', 'Hands', 'Speed With Ball','Ball Handle', 'Passing Accuracy', 'Passing Vision', 'Passing IQ','Speed', 'Acceleration', 'Vertical', 'Strength', 'Stamina', 'Hustle','Interior Defense', 'Perimeter Defense', 'Help Defense IQ','Lateral Quickness', 'Pass Perception', 'Steal', 'Block','Defensive Consistency', 'Offensive Rebound', 'Defensive Rebound','Intangibles', 'Potential']
# torch.tensor(df_ratings_NBA2k_2020_2023_PLAYER_ID.query(f"Season_year == {season_year} and Team == '{team_name}' and Player == '{player_name}'")[rating_columns].values[0])

In [135]:
class GNN_model(torch.nn.Module):
    def __init__(self,in_channels,out_channels,gfunc, dropout,num_classes):
        super(GNN_model, self).__init__()
        self.in_channels = in_channels
        self.gfunc = gfunc
        self.n_global_functions = len(self.gfunc)
        self.lin_global = Linear(in_channels * self.n_global_functions, num_classes, bias=True)
        self.dropout = dropout
        self.gnn_layer = Sequential('x, edge_index',[
            (GINConv(
                torch.nn.Sequential(
                    Linear(in_channels, in_channels),
                    BatchNorm1d(in_channels),
                    PReLU(in_channels),
                    Linear(in_channels, in_channels),
                    PReLU(in_channels),
                )
            ),'x, edge_index -> h_hat'),
            (BatchNorm1d(in_channels),'h_hat -> h_hat'),
            (Linear(in_channels, in_channels, bias=True),'h_hat -> h_hat'),
            PReLU(in_channels),
        ])
            # (GATConv(in_channels,out_channels),'h_hat -> h_hat'),
            # (GATConv(in_channels,out_channels,dropout=self.dropout),'h_hat -> h_hat'),
        # self.norm = BatchNorm1d(in_channels)
            # (BatchNorm1d(in_channels),'h_hat -> h_hat'),
        
        self.prelu = PReLU(out_channels)

        self.gat = GATConv(in_channels,in_channels,dropout=self.dropout)
    
    def readout_vector(self,h,batch,gfunc,device='cpu'):

        global_vector = torch.tensor([]).to(device)

        for g in gfunc:
            global_vector = torch.cat([global_vector,g(h,batch).to(device)], dim=1).to(device)
        return global_vector
        
    def forward(self, x, edge_index, batch, device='cpu'):
        
        alpha = .5
        h1 = self.gnn_layer(x,edge_index)
        h1 = h1 * alpha + x * (1-alpha)

        h2 = self.gat(h1,edge_index)
        h2 = h2 * alpha + h1 * (1-alpha)


        # 2. Readout layer
        h_global_mean_pool = self.readout_vector(h2,batch,self.gfunc,device=device).squeeze()

        y_pred = self.prelu(self.lin_global(h_global_mean_pool))
        y_pred = F.dropout(y_pred, p=self.dropout, training=self.training)

        return y_pred

In [136]:
season_list = []
for year in range(3):
    season_list.append('202'+str(year)+'-2'+str(year+1))

graph_dataset = MyOwnDataset(
    root='.',
    dataset_name='nba_graph_dataset_v1',
    season_list=season_list,
    df_nba_teams=df_nba_teams,
    season_campaign_all_teams_dict=season_campaign_all_teams_dict
)

graph_dataset.edge_index = add_self_loops(graph_dataset.edge_index)[0]

data_train_loader = DataLoader(
    dataset=graph_dataset,
    batch_size=30
)

In [137]:

# data_train_loader = DataLoader(G_completed_team_1_2_list,30)
# in_channels = G_completed_team_1_2_list[0].x[0].shape[0]
in_channels = 37
model = GNN_model(
    in_channels=in_channels,
    out_channels=1,
    gfunc=[global_mean_pool, global_max_pool, global_add_pool],
    dropout=.7,
    num_classes=graph_dataset.num_classes,
)

print(model)

epoch_number = 100
criterion = torch.nn.CrossEntropyLoss(reduction='mean')
lr = 5e-4
weight_decay = 5e-3
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

list_main_kpis_epoch = []
for epoch in range(1, epoch_number+1):    
    print(f'Epoch: {epoch}')
    train(
        model=model,
        data_train_loader=data_train_loader,
        criterion=criterion,
        optimizer=optimizer,
        )
    acc_test, losses_test, precision_test, precision_avg_test, recall_test, recall_avg_test, f1_score_result_test, f1_score_result_avg_test, c_matrix_test, actual_y_total_cpu, pred_y_total_cpu = test(
        model=model,
        data_train_loader=data_train_loader,
        criterion=criterion,
    )


    dict_main_kpis_epoch = {
        'acc_test':acc_test,
        'losses_test':losses_test,
        'precision_test':precision_test,
        'precision_avg_test':precision_avg_test,
        'recall_test':recall_test,
        'recall_avg_test':recall_avg_test,
        'f1_score_result_test':f1_score_result_test,
        'f1_score_result_avg_test':f1_score_result_avg_test,
        'c_matrix_test':c_matrix_test,
        'actual_y_total_cpu': actual_y_total_cpu,
        'pred_y_total_cpu': pred_y_total_cpu
    }

    print(dict_main_kpis_epoch)

    list_main_kpis_epoch.append(dict_main_kpis_epoch)

GNN_model(
  (lin_global): Linear(in_features=111, out_features=2, bias=True)
  (gnn_layer): Sequential(
    (0): GINConv(nn=Sequential(
    (0): Linear(in_features=37, out_features=37, bias=True)
    (1): BatchNorm1d(37, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=37)
    (3): Linear(in_features=37, out_features=37, bias=True)
    (4): PReLU(num_parameters=37)
  ))
    (1): BatchNorm1d(37, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=37, out_features=37, bias=True)
    (3): PReLU(num_parameters=37)
  )
  (prelu): PReLU(num_parameters=1)
  (gat): GATConv(37, 37, heads=1)
)
Epoch: 1
{'acc_test': 0.509180790960452, 'losses_test': 3.4610598246424886, 'precision_test': array([0.50632173, 0.51676122]), 'precision_avg_test': 0.5115414722063193, 'recall_test': array([0.73531073, 0.28305085]), 'recall_avg_test': 0.509180790960452, 'f1_score_result_test': array([0.5997005 , 0.36576018]), 'f1_score_r

In [138]:
pd.DataFrame(list_main_kpis_epoch)

,acc_test,losses_test,precision_test,precision_avg_test,recall_test,recall_avg_test,f1_score_result_test,f1_score_result_avg_test,c_matrix_test,actual_y_total_cpu,pred_y_total_cpu
0,0.509181,3.461060,"[0.5063217272904105, 0.516761217122228]",0.511541,"[0.7353107344632769, 0.2830508474576271]",0.509181,"[0.5997004953346389, 0.36576017521445514]",0.482730,"[[2603, 937], [2538, 1002]]","[tensor(1.), tensor(1.), tensor(1.), tensor(1....","[tensor(0.), tensor(0.), tensor(0.), tensor(1...."
1,0.498305,2.417549,"[0.4988619119878604, 0.49668141592920356]",0.497772,"[0.7429378531073446, 0.2536723163841808]",0.498305,"[0.5969133000453927, 0.33582647718773373]",0.466370,"[[2630, 910], [2642, 898]]","[tensor(1.), tensor(1.), tensor(1.), tensor(1....","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
2,0.496469,1.536603,"[0.4978058627347727, 0.4909616775126536]",0.494384,"[0.8011299435028248, 0.19180790960451977]",0.496469,"[0.6140521814441918, 0.27584806012593943]",0.444950,"[[2836, 704], [2861, 679]]","[tensor(1.), tensor(1.), tensor(1.), tensor(1....","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
3,0.504237,0.873922,"[0.5028473804100227, 0.5082781456953642]",0.505563,"[0.7483050847457627, 0.26016949152542374]",0.504237,"[0.6014986376021798, 0.34417040358744394]",0.472835,"[[2649, 891], [2619, 921]]","[tensor(1.), tensor(1.), tensor(1.), tensor(1....","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
4,0.507627,0.693666,"[0.5052003081664098, 0.5143008474576272]",0.509751,"[0.7409604519774011, 0.27429378531073445]",0.507627,"[0.6007787448465415, 0.35777450257921883]",0.479277,"[[2623, 917], [2569, 971]]","[tensor(1.), tensor(1.), tensor(1.), tensor(1....","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
...,...,...,...,...,...,...,...,...,...,...,...
95,0.502825,0.741847,"[0.5017940437746681, 0.5066401062416999]",0.504217,"[0.7901129943502825, 0.21553672316384181]",0.502825,"[0.6137809962694756, 0.3024177566389219]",0.458099,"[[2797, 743], [2777, 763]]","[tensor(1.), tensor(1.), tensor(1.), tensor(1....","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
96,0.494633,0.760340,"[0.4966143977191732, 0.48705722070844687]",0.491836,"[0.7872881355932203, 0.2019774011299435]",0.494633,"[0.6090472027972028, 0.28554313099041534]",0.447295,"[[2787, 753], [2825, 715]]","[tensor(1.), tensor(1.), tensor(1.), tensor(1....","[tensor(0.), tensor(0.), tensor(0.), tensor(1...."
97,0.500282,0.693561,"[0.5002018570851837, 0.5004703668861712]",0.500336,"[0.7, 0.30056497175141245]",0.500282,"[0.5834706851895456, 0.3755735968937522]",0.479522,"[[2478, 1062], [2476, 1064]]","[tensor(1.), tensor(1.), tensor(1.), tensor(1....","[tensor(0.), tensor(0.), tensor(0.), tensor(0...."
98,0.498588,0.698041,"[0.499098774333093, 0.49673629242819844]",0.497918,"[0.7822033898305085, 0.21497175141242938]",0.498588,"[0.609375, 0.3000788643533123]",0.454727,"[[2769, 771], [2779, 761]]","[tensor(1.), tensor(1.), tensor(1.), tensor(1....","[tensor(0.), tensor(1.), tensor(0.), tensor(1...."


In [139]:
# pd.DataFrame(list_main_kpis_epoch).iloc[9]['actual_y_total_cpu']
pd.DataFrame(pd.DataFrame(list_main_kpis_epoch).iloc[99]['actual_y_total_cpu'])[0].value_counts()

0
1.0    3540
0.0    3540
Name: count, dtype: int64

In [140]:
pd.DataFrame(pd.DataFrame(list_main_kpis_epoch).iloc[99]['pred_y_total_cpu'])[0].value_counts()

0
0.0    4905
1.0    2175
Name: count, dtype: int64